# Regular Representation

<i>Version 2</i>

## Theory

Let $G = \langle A, \circ \rangle$, be a group, where $A = \{a_0, a_1, \dots , a_{n - 1}\}$ is the set of the group's elements, and $\circ$ is its binary operator.

Also, let $B = \{\hat{b}_0, \hat{b}_1, \dots , \hat{b}_{n-1} \}$ be a set of $nx1$ vectors:

$\hat{b}_0 = \begin{bmatrix}
1 \\
0 \\
0 \\
\vdots \\
0 \end{bmatrix},
\hat{b}_1 = \begin{bmatrix}
0 \\
1 \\
0 \\
\vdots \\
0 \end{bmatrix},
\dots,
\hat{b}_{n-1} = \begin{bmatrix}
0 \\
0 \\
0 \\
\vdots \\
1 \end{bmatrix}$

And, define the following bijection between $A$ and $B$:

$V(a_i) = \hat{b}_i$ for $i = 0, \dots , n - 1$

and let $\cdot$ denote matrix-vector multiplication, and define the $nxn$ matrix,

$C_k = (c^k_{ij})_{i,j=0,\dots,n-1}$

where $c^k_{ij} = \hat{b}_i^T \cdot V(a_k \circ V^{-1}(\hat{b}_j))$

## Implementation (on a Small Test Group)

The following Python code demonstrates the generation of matrices, $C_0 \dots , C_{n - 1}$, based on a small group of order $n$.

In [1]:
import numpy as np
import finite_algebras as alg

### A Small Test Group

In [2]:
grp = alg.make_finite_algebra(
    'Z4',
    'Cyclic group of order 4',
    ['R0', 'R90', 'R180', 'R270'],
    [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]
)

grp.about()


** Group **
Name: Z4
Instance ID: 4616712848
Description: Cyclic group of order 4
Order: 4
Identity: R0
Commutative? Yes
Cyclic?: Yes
  Generators: ['R270', 'R90']
Elements:
   Index   Name   Inverse  Order
      0      R0      R0       1
      1     R90    R270       4
      2    R180    R180       2
      3    R270     R90       4
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]


In [3]:
a = grp.elements
n = grp.order

### The Bijection between Group Elements and Vectors

In [4]:
b = [np.zeros((n,1), dtype=int) for _ in range(n)]

i = 0
for x in b:
    x[i] = 1
    i += 1
    
b

[array([[1],
        [0],
        [0],
        [0]]),
 array([[0],
        [1],
        [0],
        [0]]),
 array([[0],
        [0],
        [1],
        [0]]),
 array([[0],
        [0],
        [0],
        [1]])]

In [5]:
mapping = dict(zip(a, b))
mapping

{'R0': array([[1],
        [0],
        [0],
        [0]]),
 'R90': array([[0],
        [1],
        [0],
        [0]]),
 'R180': array([[0],
        [0],
        [1],
        [0]]),
 'R270': array([[0],
        [0],
        [0],
        [1]])}

The dictionary, above, maps group elements to "vectors" (NumPy arrays). To complete the bijection, we need another dictionary that goes in the reverse direction, "vectors" to group elements. However, dictionary keys must be immutable, and NumPy arrays are mutable, so the code, below, transforms NumPy arrays to tuples, which <i>are</i> immutable.

In [6]:
def to_tuple(vec):
    '''Turns a column vector into a tuple, for use as a dictionary key.'''
    return tuple(map(lambda x: x[0], list(vec)))

inv_mapping = {to_tuple(val): key for key, val in mapping.items()}
inv_mapping

{(1, 0, 0, 0): 'R0',
 (0, 1, 0, 0): 'R90',
 (0, 0, 1, 0): 'R180',
 (0, 0, 0, 1): 'R270'}

In [7]:
def V(elem):
    '''Given a group element name, return the corresponding vector.'''
    return mapping[elem]

def Vinv(vec):
    '''Given a vector, return the corresponding element.'''
    return inv_mapping[to_tuple(vec)]

elem = a[1]
vec = V(elem)
elem2 = Vinv(vec)

print(elem)
print(vec)
print(elem2)

R90
[[0]
 [1]
 [0]
 [0]]
R90


### Regular Representation Matrices

Recall from the Theory section, above, $C_k = (c^k_{ij})_{i,j=0,\dots,n-1}$ where $c^k_{ij} = \hat{b}_i^T \cdot V(a_k \circ V^{-1}(\hat{b}_j))$

In [8]:
reg_rep = dict()

for k in range(n):
    c_k = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            c_k[i][j] = np.dot(b[i].transpose(), V(grp.op(a[k], Vinv(b[j]))))
    reg_rep[a[k]] = c_k

reg_rep

{'R0': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'R90': array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]),
 'R180': array([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 'R270': array([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])}

### Verification

The dictionary below was entered by hand, and has the correct regular representation mapping for the test group used above.

In [9]:
rot_dict = {
    'R0': np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]]),
    'R90': np.array([[0, 0, 0, 1],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]]),
    'R180': np.array([[0, 0, 1, 0],
                      [0, 0, 0, 1],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0]]),
    'R270': np.array([[0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1],
                      [1, 0, 0, 0]])
}

Test to see if all of the matrices, computed for <b>reg_rep</b>, equal their counterpart in <b>rot_dict</b>.

In [10]:
all([np.array_equal(reg_rep[elem], rot_dict[elem]) for elem in a])

True